In [ ]:
from zeep import Client
from zeep.wsse.username import UsernameToken
from base64 import b64decode
from zipfile import ZipFile
import pandas as pd
import numpy as np
import csv
import io
import json
import os, time, datetime
from datetime import datetime
from time import perf_counter

In [ ]:
pip install zeep

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
dict_creden_teryt = {
                    'username' : 'TestPubliczny',
                    'password' : '1234abcd',
                    'wsdl' : 'https://uslugaterytws1test.stat.gov.pl/wsdl/terytws1.wsdl'

                    }
dict_creden_teryt

{'username': 'TestPubliczny',
 'password': '1234abcd',
 'wsdl': 'https://uslugaterytws1test.stat.gov.pl/wsdl/terytws1.wsdl'}

In [ ]:
token = UsernameToken(
    username=dict_creden_teryt['username'],
    password=dict_creden_teryt['password']
)
type(token)

zeep.wsse.username.UsernameToken

In [ ]:
client = Client(wsdl=dict_creden_teryt['wsdl'],wsse=token)

In [ ]:
STATE_DATE = datetime.now()
catalog = client.service.PobierzKatalogTERC(STATE_DATE)
filename = catalog['nazwa_pliku']
content = catalog['plik_zawartosc']
decoded = b64decode(content)


In [ ]:
filename+('.zip')

'TERC_Urzedowy_2024-01-21.zip'

In [ ]:
with open(filename, 'wb') as file:
  file.write(decoded)
  file.close()

In [ ]:
zf = ZipFile(filename,'r')

In [ ]:
type(zf)

zipfile.ZipFile

In [ ]:
zf.namelist()

['TERC_Urzedowy_2024-01-21.xml', 'TERC_Urzedowy_2024-01-21.csv']

In [ ]:
with zf.open(zf.namelist()[1]) as csv_file:
  data = pd.read_csv(csv_file,
                     encoding='utf-8',
                     sep=';',
                     dtype={'WOJ' : str,
                            'POW' : str,
                            'GMI' : str,
                            'RODZ' : str})

data

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,NaN,NaN,NaN,DOLNOŚLĄSKIE,województwo,2024-01-01
1,02,01,NaN,NaN,bolesławiecki,powiat,2024-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2024-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2024-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2024-01-01
...,...,...,...,...,...,...,...
4327,32,61,01,1,Koszalin,gmina miejska,2024-01-01
4328,32,62,NaN,NaN,Szczecin,miasto na prawach powiatu,2024-01-01
4329,32,62,01,1,Szczecin,gmina miejska,2024-01-01
4330,32,63,NaN,NaN,Świnoujście,miasto na prawach powiatu,2024-01-01


In [ ]:
data['POW'].values

array([nan, '01', '01', ..., '62', '63', '63'], dtype=object)

In [ ]:
data_terc = data.replace(np.nan, '')
data_terc

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,,,,DOLNOŚLĄSKIE,województwo,2024-01-01
1,02,01,,,bolesławiecki,powiat,2024-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2024-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2024-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2024-01-01
...,...,...,...,...,...,...,...
4327,32,61,01,1,Koszalin,gmina miejska,2024-01-01
4328,32,62,,,Szczecin,miasto na prawach powiatu,2024-01-01
4329,32,62,01,1,Szczecin,gmina miejska,2024-01-01
4330,32,63,,,Świnoujście,miasto na prawach powiatu,2024-01-01


In [ ]:
voivodeship_df = data_terc[data_terc.NAZWA_DOD == 'województwo'].reset_index(drop=True).drop(["POW",
                                                                                              "GMI",
                                                                                              "RODZ",
                                                                                              "NAZWA_DOD"], axis=1)
voivodeship_df

,WOJ,NAZWA,STAN_NA
0,02,DOLNOŚLĄSKIE,2024-01-01
1,04,KUJAWSKO-POMORSKIE,2024-01-01
2,06,LUBELSKIE,2024-01-01
3,08,LUBUSKIE,2024-01-01
4,10,ŁÓDZKIE,2024-01-01
5,12,MAŁOPOLSKIE,2024-01-01
6,14,MAZOWIECKIE,2024-01-01
7,16,OPOLSKIE,2024-01-01
8,18,PODKARPACKIE,2024-01-01
9,20,PODLASKIE,2024-01-01


In [ ]:
for i in np.arange(0, len(voivodeship_df.columns)):
  voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].astype(str)
  voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.strip()
  voivodeship_df[voivodeship_df.columns[i]] = voivodeship_df[voivodeship_df.columns[i]].str.upper()
  print(voivodeship_df.columns[i])

WOJ
NAZWA
STAN_NA


In [ ]:
voivodeship_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   WOJ      16 non-null     object
 1   NAZWA    16 non-null     object
 2   STAN_NA  16 non-null     object
dtypes: object(3)
memory usage: 512.0+ bytes


In [ ]:
voivodeship_df.WOJ.unique()

array(['02', '04', '06', '08', '10', '12', '14', '16', '18', '20', '22',
       '24', '26', '28', '30', '32'], dtype=object)